In [0]:
import keras
import os

def restart_runtime():
    os.kill(os.getpid(), 9)

if keras.__version__ == "2.2.5":
    !pip install --upgrade keras
    restart_runtime()
  
keras.__version__

In [0]:
import os 
if not os.path.exists("auto_tag_gala"):
  !git clone https://ARprashant@bitbucket.org/ARprashant/auto_tag_gala.git
  
!cp -rf /content/auto_tag_gala/dataset /content/

In [0]:
import pandas as pd
import numpy as np
import cv2
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
train.head()

In [0]:
train['Image_path'] = train['Image'].apply(lambda x:"dataset/Train Images/"+x)
test['Image_path'] = test['Image'].apply(lambda x:"dataset/Test Images/"+x)

In [0]:
train['image_shape'] = train["Image_path"].apply(lambda x:cv2.imread(x).shape)
test['image_shape'] = test["Image_path"].apply(lambda x:cv2.imread(x).shape)

In [0]:
print("Training Images")
print("Maximum Size : ",max(train.image_shape))
print("Minimum Size : ",min(train.image_shape))

print("Testing Images")
print("Maximum Size : ",max(test.image_shape))
print("Minimum Size : ",min(test.image_shape))

In [0]:
from keras.layers import Conv2D , BatchNormalization , Dropout , Dense, Activation , Input , MaxPool2D , GlobalAvgPool2D , Flatten
from keras.models import Model
from keras.utils import to_categorical

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
encoder  = LabelEncoder()
classes = encoder.fit_transform(train.Class)

In [0]:
train['encoded_class'] = classes

In [0]:
import seaborn as sns
sns.countplot(train.encoded_class)

In [0]:
!rm -rf train_augmented val_augmented

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_val = train_test_split(train,test_size=0.2,random_state=111,shuffle=True)

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator( 
  rescale=1. / 255,
  rotation_range = 30,
  brightness_range=[0.65,1.1],
  shear_range=0.15,
  zoom_range=0.25,
  featurewise_std_normalization=True,
  horizontal_flip = True,
  vertical_flip=True,
  # samplewise_std_normalization=True
)

In [0]:
from keras.preprocessing.image import load_img , img_to_array
from numpy import expand_dims
from google.colab.patches import cv2_imshow
import os

from tqdm import tqdm

x_train_data,y_train_data,x_test_data,y_test_data =[],[],[],[]

for data,folder in list(zip([x_train,x_val],["train","val"])):
  for i ,row in data.iterrows():
    path = row['Image_path']
    image = load_img(path)
    image = image.resize((224,224))
    if folder == "train":
        x_train_data.append(img_to_array(image))
        y_train_data.append(encoder.transform([row['Class']])[0])
    else:
        x_test_data.append(img_to_array(image))
        y_test_data.append(encoder.transform([row['Class']])[0])
 
  print("Completed Createing Data")

Checking Class weights

In [0]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(train.Class),train.Class)
class_weights

In [0]:
print(np.unique(y_train_data,return_counts=True))
print(np.unique(y_test_data,return_counts=True))

In [0]:
from keras.utils import to_categorical
y_train_updated = to_categorical(y_train_data,num_classes=4)
y_test_updated =  to_categorical(y_test_data,num_classes=4)
y_train_updated.shape

In [0]:
from keras.models import Model,Sequential
from keras.layers import Dense,Dropout,BatchNormalization,Conv2D,MaxPool2D,Activation,GlobalMaxPooling2D , GlobalAvgPool2D
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet121 ,DenseNet169 , DenseNet201
from keras.applications.resnet import ResNet50 , ResNet101, ResNet152
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.resnet_v2 import ResNet101V2 ,ResNet152V2 , ResNet50V2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.layers import Concatenate
from keras.applications.nasnet import NASNetLarge ,NASNetMobile

In [0]:
base1 = Xception(include_top=False,
                 weights="imagenet",
                 input_shape=(224,224,3),
                 pooling="avg")

base2 = InceptionResNetV2(include_top=False,
                 weights="imagenet",
                 input_shape=(224,224,3),
                 pooling="avg")

base3 = InceptionV3(include_top=False,
                 weights="imagenet",
                 input_shape=(224,224,3),
                 pooling="avg")

base4 = DenseNet201(include_top=False,
                 weights="imagenet",
                 input_shape=(224,224,3),
                 pooling="avg")

base5 = NASNetMobile(include_top=False,
                 weights="imagenet",
                 input_shape=(224,224,3),
                 pooling="avg")

base6 = ResNet152V2(include_top=False,
                 weights="imagenet",
                 input_shape=(224,224,3),
                 pooling="avg")


base1.trainable = False
base2.trainable = False
base3.trainable = False
base4.trainable = False
base5.trainable = False
base6.trainable = False


input = Input(shape=(224,224,3))

output1 = base1(input)
output2 = base2(input)
output3 = base3(input)
output4 = base4(input)
output5 = base5(input)
output6 = base6(input)

def out(output):
  output = BatchNormalization()(output)
  output = Dropout(0.15)(output)

  output = Dense(1024,activation="relu")(output)
  output = BatchNormalization()(output)
  output = Dropout(0.2)(output)

  output = Dense(256,activation="relu")(output)
  output = BatchNormalization()(output)
  output = Dropout(0.2)(output)

  output = Dense(64,activation="relu")(output)
  output = BatchNormalization()(output)
  output = Dropout(0.2)(output)

  return output

output1 = out(output1)
# output1_ = Dense(4,activation="softmax")(output1)
output2 = out(output2)
output_1 = Concatenate()([output1,output2])
# output2_ = Dense(4,activation="softmax")(output2)
output3 = out(output3)
# output3_ = Dense(4,activation="softmax")(output3)
output4 = out(output4)
output_2 = Concatenate()([output3,output4])
# output4_ = Dense(4,activation="softmax")(output4)
output5 = out(output5)
# output5_ = Dense(4,activation="softmax")(output5)
output6 = out(output6)
output_3 = Concatenate()([output5,output6])
# output6_ = Dense(4,activation="softmax")(output6)
# output = output3
output = Concatenate()([output_1,output_2,output_3])
output = BatchNormalization()(output)
output = Dropout(0.2)(output)

output = Dense(4,activation="softmax")(output)

In [0]:
from keras.utils import plot_model
model=Model(input,output)
sub_model1 = Model(input,output_1)
sub_model2 = Model(input,output_2)
sub_model3 = Model(input,output_3)
model.summary()
plot_model(model)

In [0]:
# from keras.callbacks import EarlyStopping

# from keras.callbacks import ReduceLROnPlateau
# reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
#                                          factor=0.1,
#                                          patience=6,
#                                          cooldown=2,
#                                          min_lr=0.0000001,
#                                          verbose=1)


# from keras.callbacks import ModelCheckpoint, EarlyStopping ,LearningRateScheduler
# checkpoint = ModelCheckpoint("/content/drive/My Drive/models/model_combined_try_6_auto_tag_gala_images_try_2.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')
# from keras.optimizers import Adam

# optimizer = Adam(lr=0.01)
# from keras import metrics

# model.compile(loss="categorical_crossentropy",optimizer=optimizer,
#                 metrics=["acc"])

# history = model.fit_generator(
#     datagen.flow(np.array(x_train_data),y_train_updated,batch_size=128), # Training Data
#     shuffle=True,
#     verbose=1,
#     epochs=100,
#     class_weight = class_weight.compute_class_weight('balanced',np.unique(y_train_data),y_train_data),
#     callbacks=[checkpoint,reduce_learning_rate],
#     validation_data= datagen.flow(np.array(x_test_data),y_test_updated,batch_size=128)
#     )

In [0]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
!git clone https://harsh_wh@bitbucket.org/harsh_wh/modelrepo.git

In [0]:
from keras.models import load_model
model = load_model("modelrepo/model_combined_try_6_auto_tag_gala_images_try_2.h5")

In [0]:
!mkdir testing_images/

!cp -rf dataset/Test\ Images/ testing_images/test/

from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    featurewise_std_normalization=True,
    )

In [0]:
from keras.preprocessing.image import img_to_array
names =[]
predictions = [] 

for image in os.listdir("testing_images/test"):
  if os.path.isdir("testing_images/test/"+image):
    continue

  img = load_img("testing_images/test/"+image)
  img = img.resize((224,224))
  img = img_to_array(img)
  img = expand_dims(img,axis=0)
  
  names.append(image)
  prediction = model.predict_generator(test_datagen.flow(np.array(img),batch_size=1,shuffle=False))
  predictions.append(prediction[0])

In [0]:
import numpy as np
predictions_ = np.argmax(predictions,axis=1)
np.array(predictions).shape

In [0]:
np.unique(predictions_,return_counts=True)

In [0]:
final = pd.DataFrame(list(zip(names,predictions_)),columns=['Image','Class'],index=None)

In [0]:
final.Class = final.Class.apply(lambda x:encoder.inverse_transform([x])[0])

In [0]:
final.to_csv("submission.csv",index=None)

In [0]:
# predictions
pred_csv = pd.DataFrame(predictions)
pred_csv['Image'] = names
pred_csv.to_csv("combined_model_predictions_workers_with_pair_concatenation.csv",index=None)

In [0]:
sub_model1 = Model(input,output_1)
sub_model2 = Model(input,output_2)
sub_model3 = Model(input,output_3)
sub_model1.summary()

In [0]:
if not os.path.exists("sub_model1.hdf5"):
    sub_model1.save_weights("sub_model1.hdf5")
else:
    sub_model1.load_weights("sub_model1.hdf5")

if not os.path.exists("sub_model2.hdf5"):
    sub_model2.save_weights("sub_model2.hdf5")
else:
    sub_model1.load_weights("sub_model1.hdf5")

if not os.path.exists("sub_model3.hdf5"):
    sub_model3.save_weights("sub_model3.hdf5")
else:
    sub_model1.load_weights("sub_model1.hdf5")

In [0]:
pred_1_train = sub_model1.predict_generator(datagen.flow(np.array(x_train_data),batch_size=1,shuffle=False))
print("completed_prediction_1")

pred_2_train = sub_model2.predict_generator(datagen.flow(np.array(x_train_data),batch_size=1,shuffle=False))
print("completed_prediction_2")

pred_3_train = sub_model3.predict_generator(datagen.flow(np.array(x_train_data),batch_size=1,shuffle=False))
print("completed_prediction_3")

In [0]:
train_x = pd.DataFrame(np.concatenate([pred_1_train,pred_2_train,pred_3_train],axis=1))
train_x.head()
train_x['Class'] = y_train_data
train_x.to_csv("train_x.csv",index=None)

In [0]:
pred_1_test = sub_model1.predict_generator(datagen.flow(np.array(x_test_data),batch_size=1,shuffle=False))
print("completed_prediction_1")

pred_2_test = sub_model2.predict_generator(datagen.flow(np.array(x_test_data),batch_size=1,shuffle=False))
print("completed_prediction_2")

pred_3_test = sub_model3.predict_generator(datagen.flow(np.array(x_test_data),batch_size=1,shuffle=False))
print("completed_prediction_3")

In [0]:
test_x = pd.DataFrame(np.concatenate([pred_1_test,pred_2_test,pred_3_test],axis=1))
test_x.head()
test_x['Class'] = y_test_data
test_x.to_csv("test_x.csv",index=None)

In [0]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
train_x = pd.read_csv("train_x.csv")
test_x = pd.read_csv("test_x.csv")
model = XGBClassifier()
X = train_x.drop("Class",axis=1)
Y = train_x[['Class']]
X_test = test_x.drop("Class",axis=1)
Y_test = test_x[['Class']]

In [0]:
model.fit(np.array(X),Y)
model.score(np.array(X_test),Y_test)

In [0]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
params ={ 
    'booster' : ["gbtree","dart"],
    'colsample_bylevel': np.divide(list(range(80,102,5)),100),
    'colsample_bynode': np.divide(list(range(80,102,5)),100),
    'colsample_bytree': np.divide(list(range(80,102,5)),100),
    'learning_rate': [0.01,0.05,0.1,0.15,0.2,0.25,0.3],
    "gamma" :[0,0.000001,0.00001],
    'max_depth': [2,3,4,5,6,7,8],
    'n_estimators': [80,100,120,150,180,200,250,280,300,350,400,500],
    'objective': ['multi:softprob'],
    'random_state': [11],
    "min_child_weight": [0,1,5,10,15,20],
    'reg_alpha': [0,0.01,0.05,0.001,0.005],
    'reg_lambda': [0.85,0.88,0.95,0.98,1],
    'subsample': [0.8,0.85,0.9,0.95,1],
    'verbosity': [1]
    }

In [0]:
xgb_model = XGBClassifier()
random = RandomizedSearchCV(xgb_model,param_distributions=params,cv=5,n_iter=10,verbose=1,random_state=111)
random.fit(np.array(X),Y)

In [0]:
print(random.score(np.array(X_test),Y_test))
random.best_score_

In [0]:
# from sklearn.linear_model import LogisticRegression
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
# import warnings
# warnings.filterwarnings("ignore")


# params_gradient={
#     "n_estimators" : [80,100,120,200,250,280,300,350,400],
#     "subsample" : [0.85,0.8,0.9,0.92,0.95,1],
#     "min_samples_split" : [2,3,4,5,10,15],
#     "max_depth":[2,3,4,5,6],
#     "learning_rate":[0.25,0.2,0.15,0.1,0.01,0.05,0.001]
# }
# params_logisitc={
#     "max_iter" : [100,150,200,250,300,350,400,450,500]
# }

# params_lgb = {
#     "num_leaves": [100,150,200,225,250,280,300,350],
#     "max_depth": [3,4,58,10,15,-1],
#     "learnign_rate": [0.001,0.05,0.075,0.1,0.15,0.2,0.3,0.4],
#     "class_weight": ['balanced'],
#     "reg_alpha": [0.01,0.05,0.1,0.15,0.2],
#     "reg_lambda": [0.01,0.05,0.1,0.15,0.2],
#     "random_state": [111,123],
#     "subsample": [0.85,0.88,0.9,0.92,0.95,1],
#     "min_split_gain": [0,0.0001,0.0005,0.001]
# }

# gradient_model = LGBMClassifier()
# # random_gradient = RandomizedSearchCV(gradient_model,param_distributions=params_lgb,cv=5,n_iter=10,verbose=1,random_state=111)
# gradient_model.fit(X,Y)
# gradient_model.score(X_test,Y_test)

In [0]:
from keras.preprocessing.image import img_to_array
names_test =[]
predictions_test = [] 

for image in os.listdir("testing_images/test"):
    if os.path.isdir("testing_images/test/"+image):
        continue

    img = load_img("testing_images/test/"+image)
    img = img.resize((224,224))
    img = img_to_array(img)
    img = expand_dims(img,axis=0)

    names_test.append(image)

    prediction_1_test = sub_model1.predict_generator(test_datagen.flow(np.array(img),batch_size=1,shuffle=False))
    prediction_2_test = sub_model2.predict_generator(test_datagen.flow(np.array(img),batch_size=1,shuffle=False))
    prediction_3_test = sub_model3.predict_generator(test_datagen.flow(np.array(img),batch_size=1,shuffle=False))
  
    predictions_test.append(np.concatenate([prediction_1_test[0],prediction_2_test[0],prediction_3_test[0]],axis=0))

In [0]:
testing = pd.DataFrame(predictions_test)
# testing.shape
predicted_probabilities = random.predict_proba(np.array(testing))
pred_boosted_csv = pd.DataFrame(predicted_probabilities)
pred_boosted_csv['Image'] = names_test
pred_boosted_csv.to_csv("combined_model_xgboost_predictions_concatenate.csv",index=None)
testing = pd.DataFrame(predictions_test)
testing.head()
predictions_random = encoder.inverse_transform(random.predict(np.array(testing)))
pd.DataFrame(list(zip(names_test,predictions_random)),columns=['Image',"Class"],index=None).to_csv("submission_random.csv",index=None)

In [0]:
one = pd.read_csv("combined_model_predictions_workers_with_pair_concatenation.csv")
two = pd.read_csv("combined_model_xgboost_predictions_concatenate.csv")

def merge(one,two):
    joined = one.merge(two,on="Image")
  
    joined["0"] = joined.apply(lambda x:(x["0_x"]+x["0_y"])/2,axis=1)
    joined["1"] = joined.apply(lambda x:(x["1_x"]+x["1_y"])/2,axis=1)
    joined["2"] = joined.apply(lambda x:(x["2_x"]+x["2_y"])/2,axis=1)
    joined["3"] = joined.apply(lambda x:(x["3_x"]+x["3_y"])/2,axis=1)

    return joined[['0','1','2','3',"Image"]]

joined = merge(one,two)
m_predictions = np.array(joined[["0","1","2","3"]])

m_csv = pd.DataFrame(m_predictions)
m_csv['Image'] = names_test
m_csv.to_csv("model_final_all.csv",index=None)

In [0]:
predictions_ = np.argmax(m_predictions,axis=1)
final = pd.DataFrame(list(zip(names_test,predictions_)),columns=['Image','Class'],index=None)
final.Class = final.Class.apply(lambda x:encoder.inverse_transform([x])[0])
final.to_csv("final_submission.csv",index=None)